In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as ts
from init_objects import *
from functions.stylizedfacts import *
from functions.helpers import organise_data
from functions.evolutionaryalgo import quadratic_loss_function
import statsmodels.api as sm
from matplotlib import style
import json
from collections import OrderedDict
import seaborn as sns
import scipy.stats as stats
import scipy
import random

In [2]:
style.use('ggplot')

# Model dynamics

In [3]:
params = {"spread_max": 0.004087, "fundamental_value": 166, 
          "trader_sample_size": 19, "n_traders": 1000, 
          "ticks": 2500, "std_fundamental": 0.0530163128919286, 
          "std_noise": 0.10696588473846724, "w_mean_reversion": 93.63551013606137, 
          "w_fundamentalists": 8.489180919376432, "w_momentum": 43.055017297045524, 
          "max_order_expiration_ticks": 30, "std_vol": 7, "w_random": 73.28414619497076, 
          "horizon_max": 10}

## Change model to collect correct data

1. close_price, fundamental
2. agents: type, selected agents
3. bid_book orders, price, size

In [4]:
def sim_fin_model(traders, orderbook, parameters, seed=1):
    """
    The main model function
    :param traders: list of Agent objects
    :param orderbook: object Order book
    :param parameters: dictionary of parameters
    :param seed: integer seed to initialise the random number generators
    :return: list of simulated Agent objects, object simulated Order book
    """
    close_price = []
    fundamental_val = []
    volume = []
    
    random.seed(seed)
    np.random.seed(seed)
    fundamental = [parameters["fundamental_value"]]

    for tick in range(parameters['horizon_max'] + 1, parameters["ticks"]):
        # evolve the fundamental value via random walk process
        fundamental.append(fundamental[-1] + parameters["std_fundamental"] * np.random.randn())

        # select random sample of active traders
        active_traders = random.sample(traders, int((parameters['trader_sample_size'])))

        # update common expectation components
        mid_price = orderbook.tick_close_price[-1]
        fundamental_component = np.log(fundamental[-1] / mid_price)
        chartist_component = np.cumsum(orderbook.returns[:-parameters['horizon_max']-1:-1]
                                       ) / np.arange(1., float(parameters['horizon_max'] + 1))

        for trader in active_traders:
            # update trader specific expectations
            noise_component = parameters['std_noise'] * np.random.randn()

            fcast_return = trader.var.forecast_adjust * (
                trader.var.weight_fundamentalist * fundamental_component +
                trader.var.weight_chartist * chartist_component[trader.par.horizon] +
                trader.var.weight_random * noise_component -
                trader.var.weight_mean_reversion * chartist_component[trader.par.horizon])

            fcast_price = mid_price * np.exp(fcast_return)

            # submit orders
            if fcast_price > mid_price:
                bid_price = fcast_price * (1. - trader.par.spread)
                orderbook.add_bid(bid_price, abs(int(np.random.normal(scale=parameters['std_vol']))), trader)
            elif fcast_price < mid_price:
                ask_price = fcast_price * (1 + trader.par.spread)
                orderbook.add_ask(ask_price, abs(int(np.random.normal(scale=parameters['std_vol']))), trader)

        # match orders in the order-book
        while True:
            matched_orders = orderbook.match_orders()
            if matched_orders is None:
                break

        # clear and update order-book history
        volume.append(sum(orderbook.transaction_volumes))
        orderbook.cleanse_book()
        fundamental_val.append(fundamental[-1])
        orderbook.fundamental = fundamental
        close_price.append(orderbook.tick_close_price[-1])
        

    return traders, orderbook, close_price, fundamental_val, volume


## Run model

In [5]:
NRUNS = 1
traders = []
obs = []
for seed in range(NRUNS): 
    traders, orderbook = init_objects(params, seed)
    traders, orderbook, close_price, fundamental_val, volume = sim_fin_model(traders, orderbook, params, seed)
    traders.append(traders)
    obs.append(orderbook)

In [6]:
#close_price 
#fundamental_val
#volume

## Create graph of prices, fundamental, vol

In [7]:
%matplotlib tk
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

In [25]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,9))

ax1.plot(range(len(close_price)), fundamental_val, label='fundamental value')
ax1.plot(range(len(close_price)), close_price, label='close price')

ax1.set_ylabel('End tick price', fontsize='14')
ax1.legend(loc='best', fontsize='14')

ax2.bar(range(len(volume)), volume, width=1.0)
ax2.set_ylabel('Volume', fontsize='14')
ax2.set_xlabel('Ticks', fontsize='14')

# for ax in [ax1, ax2]:
#     ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

Text(0.5,0,'Ticks')

## Animate

In [42]:
animation.writers.avail

{u'html': matplotlib.animation.HTMLWriter,
 u'pillow': matplotlib.animation.PillowWriter}

In [97]:
# Set up formatting for the movie files
Writer = animation.writers['pillow']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

In [113]:
animation_time = 1000

def data_gen(t=0):
    cnt = 0
    while t < animation_time:
        cnt += 1
        t += 1
        yield t, close_price[cnt], fundamental_val[cnt]


def init():
    ax.set_ylim(150, 180)
    ax.set_xlim(0, animation_time)
    
    del xdata[:]
    del ydata[:]
    del fdata[:]
    line.set_data(xdata, ydata)
    line2.set_data(xdata, fdata)
    ax.legend((line, line2), ('close_price', 'fundamental'), fontsize='14')
    return line,

fig, (ax, ax2) = plt.subplots(2, 1)
line, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)
ax.grid()
xdata, ydata, fdata = [], [], []


def run(data):
    # update the data
    t, y, fun = data
    xdata.append(t)
    ydata.append(y)
    fdata.append(fun)
    line.set_data(xdata, ydata)
    line2.set_data(xdata, fdata)

    return line,

ani = animation.FuncAnimation(fig, run, data_gen, blit=False, interval=1,
                              repeat=False, init_func=init, save_count=animation_time)

#ani.save('price_fundamental.gif', writer=writer)
plt.show()

## Volume

In [ ]:
animation_time = 1000

def data_gen(t=0):
    cnt = 0
    while t < animation_time:
        cnt += 1
        t += 1
        yield t, close_price[cnt], fundamental_val[cnt]


def init():
    ax.set_ylim(150, 180)
    ax.set_xlim(0, animation_time)
    
    del xdata[:]
    del ydata[:]
    del fdata[:]
    line.set_data(xdata, ydata)
    line2.set_data(xdata, fdata)
    ax.legend((line, line2), ('close_price', 'fundamental'), fontsize='14')
    return line,

fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)
ax.grid()
xdata, ydata, fdata = [], [], []


def run(data):
    # update the data
    t, y, fun = data
    xdata.append(t)
    ydata.append(y)
    fdata.append(fun)
    line.set_data(xdata, ydata)
    line2.set_data(xdata, fdata)

    return line,

ani = animation.FuncAnimation(fig, run, data_gen, blit=False, interval=1,
                              repeat=False, init_func=init, save_count=animation_time)

ani.save('price_fundamental.gif', writer=writer)